In [1]:
import google.generativeai as genai
from google.colab import userdata
gemini_key = userdata.get('gemini_key')


genai.configure(api_key=gemini_key)

In [2]:
# configuration du modèle
generation_config = {
  "temperature": 0,
  "top_p": 1,
  "top_k": 5,
  "max_output_tokens": 2048,
}

In [3]:
# appel au modèle generative avec la configuration déjà fixée
model = genai.GenerativeModel(model_name= "gemini-1.0-pro",
                              generation_config = generation_config)

In [6]:
import csv
import random
import pandas as pd

# Charger le fichier TSV
df = pd.read_csv("/content/cti-mcq.tsv", sep="\t")

# Sélectionner les colonnes souhaitées
columns_to_extract = ["Question", "Option A", "Option B", "Option C", "Option D","Prompt	GT"]
df_questions = df[columns_to_extract]

# Extraire aléatoirement 100 lignes
df_questions_sample = df_questions.sample(n=100)

# Enregistrer les questions dans un fichier CSV
df_questions_sample.to_csv("questions.csv", index=False)

# Extraire les valeurs GT correspondantes
df_gt = df.loc[df_questions_sample.index, "GT"]

# Enregistrer les valeurs GT dans un fichier CSV
df_gt.to_csv("gt.csv", index=False)

In [ ]:
# Ouvrir un fichier CSV pour enregistrer les réponses
with open("responses.csv", 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Question", "Option A", "Option B", "Option C", "Option D", "Réponse"])  # En-tête

    # Charger les questions à partir de "questions.csv"
    with open("questions.csv", 'r', newline='') as f:
        reader = csv.DictReader(f)
        for row in reader:
            question = row["Question"]
            option_a = row["Option A"]
            option_b = row["Option B"]
            option_c = row["Option C"]
            option_d = row["Option D"]
            #bonne_reponse = row["GT"]  # Assuming "GT" is the column for the correct answer

            # Générer le prompt
            prompt = f"Répond brièvement à la question suivante en choisissant la bonne réponse parmi les options A,B?C ou D : \n\n{question}\n\nA. {option_a}\nB. {option_b}\nC. {option_c}\nD. {option_d}"

            response = model.generate_content(prompt, stream=True)




In [8]:
# Ouvrir le fichier en mode lecture
fichier = open('gt.csv', 'r')

# Lire le contenu du fichier
contenu = fichier.read()

# Afficher le contenu
print(contenu)

# Fermer le fichier
fichier.close()

GT
C
C
C
C
C
A
C
A
C
B
C
C
D
C
B
B
A
C
C
C
C
C
D
C
B
B
D
C
D
C
A
C
C
B
C
A
A
A
C
C
B
B
A
C
B
A
A
C
C
D
B
C
B
B
B
A
C
B
C
B
A
A
B
A
C
A
C
C
D
A
C
B
C
B
D
A
C
C
B
C
D
B
B
B
A
C
B
B
C
B
C
C
A
C
B
D
B
B
D
B



In [11]:
# Ouvrir le fichier en mode lecture
fichier = open('responses.csv', 'r')

# Lire le contenu du fichier
contenu = fichier.read()

# Afficher le contenu
print(contenu)

# Fermer le fichier
fichier.close()

Question,Option A,Option B,Option C,Option D,Réponse
What is the typical severity level for the attack pattern CAPEC-75: Manipulating Writeable Configuration Files?,Low,Medium,Very High,High,D. High
"Jim works as a security analyst in a large multinational company. Recently, a group of hackers penetrated into their organizational network and used a data staging technique to collect sensitive data. They collected all sorts of sensitive data about the employees and customers, business tactics of the organization, financial information, network infrastructure information and so on. What should Jim do to detect the data staging before the hackers exfiltrate from the network?",Jim should identify the attack at an initial stage by checking the content of the user agent field.,"Jim should analyze malicious DNS requests, DNS payload, unspecified domains, and destination of DNS requests.","Jim should monitor network traffic for malicious file transfers, file integrity monitoring, and event logs

In [ ]:
def calculer_score(gt_file, responses_file):
    nb_correct = 0
    nb_total = 0

    with open(gt_file, 'r', newline='') as gt_csvfile, open(responses_file, 'r', newline='') as responses_csvfile:
        gt_reader = csv.reader(gt_csvfile)
        responses_reader = csv.reader(responses_csvfile)

        # Ignorer les en-têtes
        next(gt_reader)
        next(responses_reader)

        for gt_row, responses_row in zip(gt_reader, responses_reader):
            # Extraire la bonne réponse du fichier 'gt.csv'
            bonne_reponse = gt_row[0]

            # Extraire la réponse du fichier 'responses.csv'
            reponse = responses_row[-1]

            # Vérifier si le premier caractère de la réponse est correct
            if reponse[0] == bonne_reponse:
                nb_correct += 1
            nb_total += 1

    score = (nb_correct / nb_total) * 100
    return score

# Exemple d'utilisation
gt_file = "gt.csv"
responses_file = "responses.csv"

score = calculer_score(gt_file, responses_file)
print(f"Le score de précision est de {score:.2f}%")

Le score de précision est de 43.75%
